In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from os import path
import itertools

Using TensorFlow backend.


In [2]:
A, X, E, y = qm9.load_data(return_type='numpy',
                          nf_keys='atomic_num',
                          ef_keys='type',
                          self_loops=True,
                          amount=4000)
uniq_X = np.unique(X)
X = label_to_one_hot(X, uniq_X)

Loading QM9 dataset.
Reading SDF


100%|██████████| 4000/4000 [00:00<00:00, 5000.62it/s]


In [3]:
tasks = list(y.columns)[1:]
y_list = []
for task in tasks:
    y_list.append(y[[task]].values)
for i in range(len(y_list)):
    y_list[i] = StandardScaler().fit_transform(y_list[i]).reshape(-1, y_list[0].shape[-1])

In [4]:
print(tasks)

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']


In [6]:
N = X.shape[-2]
F = X.shape[-1] 
S = E.shape[-1]
n_out = y_list[0].shape[-1]
learning_rate = 1e-3
epochs = 3
batch_size = 64
es_patience = 5

In [7]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [8]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
*y_train_test_list = train_test_split(A, X, E, *y_list, test_size = 0.1)

y_train_list = y_train_test_list[::2]
y_test_list = y_train_test_list[1::2]

In [9]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

In [10]:
def create_single_task_model(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense)
    return Model(inputs=[X_in, A_in, E_in], outputs=output)

In [11]:
def generate_filename(task):
    return path.join('single_task_trained_models', task + '.h5')

In [12]:
# for i in range(-2, 0):
#     print('learning', tasks[i])
#     model = create_single_task_model(X_in, A_in, E_in)
#     model.compile(optimizer=optimizer, loss=loss)
#     es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
#     model.fit([X_train, A_train, E_train],
#              y_train_list[i],
#              batch_size=batch_size,
#              validation_split=0.1,
#              epochs=epochs,
#              callbacks=[es_callback])
#     model.save_weights(generate_filename(tasks[i]))

learning h298_atom




Train on 3240 samples, validate on 360 samples
Epoch 1/3
3240/3240 [==============================] - 113s 35ms/step - loss: 0.6269 - val_loss: 0.3969
Epoch 2/3
3240/3240 [==============================] - 101s 31ms/step - loss: 0.2958 - val_loss: 0.1821
Epoch 3/3
3240/3240 [==============================] - 101s 31ms/step - loss: 0.1464 - val_loss: 0.1321
learning g298_atom
Train on 3240 samples, validate on 360 samples
Epoch 1/3
3240/3240 [==============================] - 104s 32ms/step - loss: 0.5973 - val_loss: 0.3942
Epoch 2/3
3240/3240 [==============================] - 106s 33ms/step - loss: 0.2432 - val_loss: 0.1459
Epoch 3/3
3240/3240 [==============================] - 102s 31ms/step - loss: 0.1242 - val_loss: 0.0966


In [13]:
def load_and_compile_model(task):
    model = create_single_task_model(X_in, A_in, E_in)
    model.load_weights(generate_filename(task))
    model.compile(optimizer=optimizer, loss=loss)

In [14]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = create_single_task_model(X_in, A_in, E_in)
    model_transferred = create_single_task_model(X_in, A_in, E_in)
    model_learned.load_weights(generate_filename(task_learned))
    model_transferred.load_weights(generate_filename(task_transferred))
    
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()    
    layers_transferred = layers_learned[:10] + layers_transferred[10:]
    model_transferred.set_weights(layers_transferred)
    model_transferred.compile(optimizer=optimizer, loss=loss)
    
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results

In [15]:
def load_and_test_model(task_learned, y_test):
#     model_learned = load_model(path.join('single_task_trained_models', task_learned+'.h5'), 
#                                custom_objects={'EdgeConditionedConv':EdgeConditionedConv,
#                                               'GlobalAttentionPool':GlobalAttentionPool})
    model = create_single_task_model(X_in, A_in, E_in)
    model.load_weights(path.join('single_task_trained_models', task_learned + '.h5'))
    model.compile(optimizer=optimizer, loss=loss)
    eval_results = model.evaluate([X_test, A_test, E_test], y_test, batch_size=batch_size)
    return eval_results

In [16]:
print(load_and_test_model('A', y_test_list[0]))
print(calculate_transfer_coefficient_from_file('A', 'B', y_test_list[1]))

400/400 [==============================] - 1s 2ms/step
4.524567862972617e-05
400/400 [==============================] - 1s 2ms/step
6.229245293140411


In [ ]:
print(tasks)
for i, j in itertools.permutations(range(len(tasks)), 2):
    task_learned = tasks[i]
    task_transferred = tasks[j]
    print(task_learned, task_transferred)
    transfer_coefficient = calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_test_list[j])
    print(transfer_coefficient)

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']
A B
400/400 [==============================] - 2s 4ms/step
6.229245293140411
A C
400/400 [==============================] - 2s 4ms/step
5.486166998147964
A mu
400/400 [==============================] - 2s 4ms/step
1.0761423802375794
A alpha
400/400 [==============================] - 2s 4ms/step
1.1863147258758544
A homo
400/400 [==============================] - 2s 4ms/step
1.0845456171035766
A lumo
400/400 [==============================] - 2s 4ms/step
1.0579222297668458
A gap
400/400 [==============================] - 2s 4ms/step
1.0387437772750854
A r2
400/400 [==============================] - 2s 4ms/step
0.9354163122177124
A zpve
400/400 [==============================] - 2s 4ms/step
1.10608829498291
A u0
400/400 [==============================] - 2s 4ms/step
1.1490273976325989
A u298
400/400 [==============================] - 2s

400/400 [==============================] - 3s 8ms/step
1.1864917278289795
homo u0
400/400 [==============================] - 3s 8ms/step
1.1609279608726502
homo u298
400/400 [==============================] - 3s 9ms/step
1.159862494468689
homo h298
400/400 [==============================] - 3s 8ms/step
1.2046837949752807
homo g298
400/400 [==============================] - 3s 8ms/step
1.13497620344162
homo cv
400/400 [==============================] - 3s 8ms/step
1.0556876754760742
homo u0_atom
400/400 [==============================] - 3s 8ms/step
1.1748163795471191
homo u298_atom
400/400 [==============================] - 3s 8ms/step
1.1738701629638673
homo h298_atom
400/400 [==============================] - 3s 8ms/step
1.1509309482574464
homo g298_atom
400/400 [==============================] - 3s 8ms/step
1.0908369541168212
lumo A
400/400 [==============================] - 3s 8ms/step
0.006239830292761326
lumo B
400/400 [==============================] - 3s 8ms/step
6.462703173160